In [11]:
import sys
import numpy as np
from gym_power.envs.active_network_env import ActiveEnv

# Tutorial showing the functionality of `ActiveEnv`
This notebook gives an introduction to the class ActiveEnv. The class follows the structure of a general `gym` environment, and can be used for reinforcement algorithms availble in [stable-baselines](https://github.com/hill-a/stable-baselines).

The agent is allowed to modify the power consumption at nodes in a distribution system with high peak demand and high production from solar power. The goal of the agent is to reduce the number of current and voltage violations in the grid by increasing/ decreasing the consumption at appropriate times. The increase/decrease in power consumption is intended to be a simplified program for demand response that exploits the residential flexbility in the grid. 

Currently, the implementation is based on a significant simplification:
- <b>Constant flexbility:</b>
The flexbility is assumed to be constant. If the agent increases the consumption in one hour, it has no consequences for the future flexibility in the system. This is not realstic, as increasing the consumption corresponds to turning on electrical equipment, and that equipment can't be turned on again later.


## Power system
<img src= "figures\cigre_network_mv_der.png" style=width:600px;height:600px;>

The electrical power grid used is constructed by the International Council on Large Electric Systems (CIGRE). There is wind power connected to bus bar 7 in the figure above, but this is for simplicity assumed to be solar. In other words, all power production from renewable sources in this net follows solar irradiance in this net. The environment can also be given another `pandapower` network.

The network is an attribute in the `ActiveEnv` class:
 


In [23]:
env = ActiveEnv(seed=3)
net = env.powergrid
net

This pandapower network includes the following parameter tables:
   - bus (15 elements)
   - load (18 elements)
   - sgen (9 elements)
   - switch (8 elements)
   - ext_grid (1 element)
   - line (15 elements)
   - trafo (2 elements)
   - bus_geodata (15 elements)
 and the following results tables:
   - res_bus (15 elements)
   - res_line (15 elements)
   - res_trafo (2 elements)
   - res_ext_grid (1 element)
   - res_load (18 elements)
   - res_sgen (9 elements)

`pandapower` stores information in Pandas DataFrames, and there is a table for each of the listed elements above. For instance, the load table  

In [24]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,Load R1,1,14.99400,3.044662,0.0,0.0,15.300,1.0,True,None
1,Load R3,3,0.27645,0.069285,0.0,0.0,0.285,1.0,True,None
2,Load R4,4,0.43165,0.108182,0.0,0.0,0.445,1.0,True,None
3,Load R5,5,0.72750,0.182329,0.0,0.0,0.750,1.0,True,None
4,Load R6,6,0.54805,0.137354,0.0,0.0,0.565,1.0,True,None
5,Load R8,8,0.58685,0.147078,0.0,0.0,0.605,1.0,True,None
6,Load R10,10,0.47530,0.119121,0.0,0.0,0.490,1.0,True,None
7,Load R11,11,0.32980,0.082656,0.0,0.0,0.340,1.0,True,None
8,Load R12,12,14.99400,3.044662,0.0,0.0,15.300,1.0,True,None
9,Load R14,14,0.20855,0.052268,0.0,0.0,0.215,1.0,True,None


# State space
<img src= "figures\observation.png" style=width:400px;height:300px;>

There are several possible state spaces that can be used to numerically represent the environment. 



## Solar and demand forecast
The state can include the solar and consumption forecast in the system. The solar forecast is generated from satellite-derived solar irradiance in central Norway. The next cell shows how to set the state space to include only the solar forecast. The attribute `env.observation_space` gives information about the state space

In [112]:
env = ActiveEnv()
env.set_parameters({'state_space':['sun']})
env.observation_space

Box(4,)

In this case shape of the state is 4 because the forecast by default is 4-hour long. This can be changed by using the `set_parameters` method

In [113]:
env.set_parameters({'forecast_horizon':24})
env.observation_space

Box(24,)

The solar irradiance is assumed to be equal everywhere in the net, so there is not a unique forecast for each bus bar. However, the solar forecast is scaled up by the nominal values, so the absolute production differs from load to load.

No forecasts are perfect, so the actual values should deviate from the forecasted values. The actual values are found by adding a noise term to the forecast. The noise term follows a Gaussian distribution with mean 0 and a standard deviation that is proportional to the forecast. The standard deviation in the forecasts are by default 3%. The unncertainty can be changed:

In [114]:
env.set_parameters({'solar_std':0.1,
                   'demand_std':0.3})

## Bus state
It is possible to include the bus state in the state representation. The bus state includes the voltage magnitude, voltage angle, active effect and reactive effect of each bus in the system. The bus values are found in the `res_bus` DataFrame 

In [111]:
env.powergrid.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.030000,0.000000,-43.196502,-15.696169
1,0.994133,-6.045087,19.839000,4.637136
2,0.977804,-6.573027,0.000000,0.000000
3,0.951848,-7.414877,0.481700,0.208882
4,0.950094,-7.508922,0.411650,0.108182
5,0.948893,-7.573678,0.697500,0.182329
6,0.947488,-7.649919,0.518050,0.137354
7,0.952199,-7.140044,-1.423500,0.047410
8,0.949187,-7.414180,0.556850,0.147078
9,0.948264,-7.449484,0.543750,0.355578


In [110]:
env = ActiveEnv()
env.set_parameters({'state_space':['bus']})
env.observation_space

Box(60,)

The size of the state space is 4 times the number of buses

## Imbalance state
It is possible to include the bus state in the state representation. The bus state includes the voltage magnitude, voltage angle, active effect and reactive effect of each bus in the system. The bus values are found in the `res_bus` DataFrame 

# Action space

<img src= "figures\N64-controller-white.jpg" style=width:300px;height:300px;>


There is one action for each row in load table of the network. The agent can independently change the consumption at each load in an interval of flexibility, for instance by +/- 10 %. The loads are assumed to have a constant power factor. In other words, if the active power increases by 10 %, then the reactive power also increases by 10 %. For the Cigre network, we have the action space $\mathcal{A} = \{a_{i}|\;i=1,...18\}$, $a_{i} \in [-1,1]$. The action is then scaled by the flexbility (0-100%), which determines the change in consumption at each load

The action space is described in `env.action_space`

In [102]:
env.action_space

Box(18,)

Concretely, `ActiveEnv` takes in the action vector $\in \mathbb{R^{18}}$, and manipulates the `net.load` DataFrame, and solves the powerflow equation. In this example, each load increases its load as much as possible. First be create an environment instance, and look at the demand forecast. The uncertainty in the forecast is set to 0. 

In [105]:
env = ActiveEnv(seed=3)
env.set_parameters({'demand_std':0})
forecast = env.demand_forecasts[:,0]
print('forecasted demand: ', forecast)

forecasted demand:  [0.45447803]


Next, we create the action vector of ones, and execute the action by calling the `step` method

In [106]:
a = np.ones(env.action_space.shape)
env.step(a)
print(a)




[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


The values for consumption and production in the net are now updated. The consumption as a percentage of nominal values is equal at each load

In [107]:
consumption = env.powergrid.load['p_mw']/env.powergrid.load['sn_mva']
consumption

0     0.499926
1     0.499926
2     0.499926
3     0.499926
4     0.499926
5     0.499926
6     0.499926
7     0.499926
8     0.499926
9     0.499926
10    0.499926
11    0.499926
12    0.499926
13    0.499926
14    0.499926
15    0.499926
16    0.499926
17    0.499926
dtype: float64

The consumption is 10 % higher than the forecast at every load because the flexibility is 10 %

In [108]:
consumption/forecast

0     1.1
1     1.1
2     1.1
3     1.1
4     1.1
5     1.1
6     1.1
7     1.1
8     1.1
9     1.1
10    1.1
11    1.1
12    1.1
13    1.1
14    1.1
15    1.1
16    1.1
17    1.1
dtype: float64

### Smaller action space
18 free variables in an action space is quite large. It is possible to set a global action that modifies all the loads in the interval of flexibility. This is done shown in the cell below. Note that the change is a percentage change in consumption and the loads have different nominal consumption levels. Therefore, the absolute power change varies from load to load. 

In [5]:
env.set_parameters({'one_action':True})
env.action_space

Box(1,)